<a href="https://colab.research.google.com/github/mirsadra/casereport/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Required Packages  
The notebook is based on Google Colab.

In [1]:
!pip install biopython


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.7 MB/s eta 0:00:00


In [2]:
from Bio import Entrez
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files

# Fetch Data From PubMed

In [10]:
def search(query):
    Entrez.email = 'miirsadra@gmail.com'
    handle = Entrez.esearch(db='pubmed',
                            sort='publication date',
                            retmax='250000',
                            retmode='xml',
                            term=query)
    results = Entrez.read(handle)
    return results

def fetch_details(id_list):
    ids = ','.join(id_list)
    Entrez.email = 'miirsadra@gmail.com'
    handle = Entrez.efetch(db='pubmed',
                           retmode='xml',
                           id=ids)
    results = Entrez.read(handle)
    return results

def fetch_data_for_query(query):
    studies = search(query)
    studiesIdList = studies['IdList']

    # Lists initialization
    author_list = []
    title_list = []
    abstract_list = []
    language_list = []
    pubdate_year_list = []
    keywords_list = []
    article_type_list = []

    chunk_size = 10000

    for chunk_i in range(0, len(studiesIdList), chunk_size):
        chunk = studiesIdList[chunk_i:chunk_i + chunk_size]
        papers = fetch_details(chunk)
        for paper in papers['PubmedArticle']:
        # Authors
            try:
                authors = paper['MedlineCitation']['Article']['AuthorList']
                if len(authors) > 1:
                    author_list.append(authors[0]['LastName'] + " et al.")
                else:
                    author_list.append(authors[0]['LastName'])
            except:
                author_list.append('No Author')

            # Title
            title_list.append(paper['MedlineCitation']['Article']['ArticleTitle'])

            # Abstract
            try:
                abstract_list.append(paper['MedlineCitation']['Article']['Abstract']['AbstractText'][0])
            except:
                abstract_list.append('No Abstract')

            # Language
            language_list.append(paper['MedlineCitation']['Article']['Language'][0])

            # Year
            try:
                pubdate_year_list.append(paper['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']['Year'])
            except:
                pubdate_year_list.append('No Data')

            # Keywords
            try:
                keywords = paper['MedlineCitation']['KeywordList']
                keywords_list.append(", ".join([kw for kw in keywords[0]]))
            except:
                keywords_list.append('No Keywords')

            # Article Type
            try:
                article_type_list.append(paper['MedlineCitation']['Article']['PublicationTypeList'][0])
            except:
                article_type_list.append('No Type')

        return pd.DataFrame(list(zip(author_list, title_list, abstract_list, language_list, pubdate_year_list, keywords_list, article_type_list)),
                        columns=['Authors', 'Title', 'Abstract', 'Language', 'Year', 'Keywords', 'Type'])

In [11]:
# Query 1
query1 = 'L-shaped renal ectopia OR L-shaped crossed renal ectopia OR L-shaped crossed fused AND Case Reports[PT]'
df1 = fetch_data_for_query(query1)

# Query 2
query2 = 'single ureter AND crossed-fused AND Case Reports[PT]'
df2 = fetch_data_for_query(query2)

# Concatenate results into a single dataframe
df = pd.concat([df1, df2], ignore_index=True)

In [12]:
df

,Authors,Title,Abstract,Language,Year,Keywords,Type
0,Kumar et al.,Primary Chondrosarcoma in L-shaped Crossed Fus...,Primary renal chondrosarcomas are rare tumors ...,eng,2022,"chondrosarcoma, kidney, renal ectopia, urinary...",Case Reports
1,Kawatani et al.,Endovascular aortic aneurysm repair with rever...,Crossed fused renal ectopia is a very rare con...,eng,2021,No Keywords,Case Reports
2,Jallouli et al.,About a case of ureteropelvic junction obstruc...,No Abstract,eng,2018,"Ectopia, Kidney, Pyloplasty",Case Reports
3,Bakov et al.,[Case report of rare co-occurrence of renal ce...,L-shaped kidney refers to a rare anomaly of th...,rus,2017,"L-shaped kidney, cancer of the renal pelvis, c...",Case Reports
4,Iwanaga et al.,Anatomical and Radiological Analyses of L-shap...,L-shaped kidney is a congenital anomaly. The d...,eng,2018,"L-shaped kidney, crossed fused renal ectopia, ...",Case Reports
5,Seo et al.,Crossed fused renal ectopia in a Persian cat.,This report describes a rare case of crossed f...,eng,2017,No Keywords,Case Reports
6,Kumar et al.,Robot-assisted heminephrectomy for chromophobe...,Renal cell carcinoma associated with fused ect...,eng,2015,"Nephrectomy, Renal cell carcinoma, Robotics",Case Reports
7,Liu et al.,"Right-Crossed, Fused Renal Ectopia L-Shaped Ki...",Crossed fused renal ectopia combined with chyl...,eng,2015,No Keywords,Case Reports
8,Nakai et al.,Laparoscopic sigmoid colectomy for a patient w...,Crossed-fused renal ectopia (CFRE) is a very r...,eng,2015,"Colon cancer, Crossed renal ectopia, Laparosco...",Case Reports
9,Pupca et al.,Left crossed fused renal ectopia L-shaped kidn...,Crossed fused renal ectopia (CFRE) is the seco...,eng,2014,No Keywords,Case Reports
